# Regression Model

## Introduction
In today's modern age, technological devices have become an integral part of society. With ever-evolving computer components, it can be difficult to conduct research on a laptop that fits an individual's specific needs. In our algorithm, we will be providing price estimates based on the hardware specifications provided in our dataset. For instance, we could use variables such as CPU speed and RAM size to predict the price of a laptop. Ultimately, our goal is to assist the users and companies by providing a price estimate for their ideal laptop, thus, reducing the time needed for research. Thus our predictive question would be **"What will be the price of a laptop based on its specifications?"**. The dataset we will be using is an open-source file from Kaggle. Link for the original dataset: https://www.kaggle.com/datasets/ehtishamsadiq/uncleaned-laptop-price-dataset/data'

## Preparing Libraries and Setup

Unfortunately, the python language does not natively support everything we will use to create this model, and it would be inefficient to re-invent the wheel. As such, we will be using the following libraries in our code:
#### **Pandas**
##### We use the DataFrame object from pandas to store and manipulate our data

#### **Altair**
##### Visualization library that allows us to make charts to help see the data

#### **Numpy**
##### Math library. We use it to set seeds and convert number Types. Also used by pandas in many operations

#### **Scikit learn**
##### Machine learning library. This will be the backbone of our regression model. We use it to build and train the model on our data, and to test the results.
 
#### **Matplotlib**
##### (insert something here)



In [1]:
### Uncomment cell below whenever Altair stops working to reinstall latest version

## For some reason, whenever the jupyter server restarts, it
## sends you back to the old version of altair (4.2.2)

In [2]:
# pip install -U altair      #<---- UNCOMMENT

In [3]:
## If the text below says anything below version 5.0.0,
## run the code above
import altair as alt; alt.__version__

'4.2.2'

In [4]:
### Run this cell before continuing.

import altair as alt
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn import set_config
from sklearn.model_selection import GridSearchCV, cross_validate, train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error


# Simplify working with large datasets in Altair
alt.data_transformers.disable_max_rows()

# Output dataframes instead of arrays
set_config(transform_output="pandas")
    
np.random.seed(1137110237) #Randomly picked seed

### Importing and Wrangling Data

(INSERT DESCRIPTION)
<br> Link for the original dataset: https://www.kaggle.com/datasets/ehtishamsadiq/uncleaned-laptop-price-dataset/data'

In [5]:
# Loading csv file data as a pandas dataframe
laptop_data = pd.read_csv("https://raw.githubusercontent.com/fyip3/ds_project/main/data/laptopData.csv")

# Cleaning data
laptop_data = laptop_data.drop(columns=["Unnamed: 0"])          # Filtering Columns
laptop_data = laptop_data.dropna()                              # Removing redundant non-numeric part
laptop_data['Ram'] = laptop_data['Ram'].str.extract('(\d+)', expand=False)
laptop_data['Weight'] = laptop_data['Weight'].str.removesuffix("kg")
laptop_data['Memory'] = laptop_data['Memory'].str.extract('(\d+)', expand=False)
laptop_data["Price"] = laptop_data["Price"] * 0.017                         # Convert Price from INR to CAD
laptop_data = laptop_data.rename(columns={"Inches": "ScreenSize_Inches", "Ram": "Memory_GB", "Memory" : "Storage", "Weight" : "Weight_Kg", "Price" : "Price_CAD"})
# Convert columns from strings to int/float
laptop_data["Memory_GB"] = pd.to_numeric(laptop_data.Memory_GB, errors='coerce')
laptop_data["Weight_Kg"] = pd.to_numeric(laptop_data.Weight_Kg, errors='coerce')
laptop_data["ScreenSize_Inches"] = pd.to_numeric(laptop_data.ScreenSize_Inches, errors='coerce')
laptop_data["Storage"] = pd.to_numeric(laptop_data.Storage, errors='coerce')
laptop_data.dtypes
count = laptop_data.nunique()  #number of unique datapoints for each varaible
count
laptop_data = laptop_data.dropna()

In [6]:
laptop_data

,Company,TypeName,ScreenSize_Inches,ScreenResolution,Cpu,Memory_GB,Storage,Gpu,OpSys,Weight_Kg,Price_CAD
0,Apple,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 2.3GHz,8,128.0,Intel Iris Plus Graphics 640,macOS,1.37,1213.437614
1,Apple,Ultrabook,13.3,1440x900,Intel Core i5 1.8GHz,8,128.0,Intel HD Graphics 6000,macOS,1.34,814.223894
2,HP,Notebook,15.6,Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,8,256.0,Intel HD Graphics 620,No OS,1.86,520.812000
3,Apple,Ultrabook,15.4,IPS Panel Retina Display 2880x1800,Intel Core i7 2.7GHz,16,512.0,AMD Radeon Pro 455,macOS,1.83,2298.320712
4,Apple,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 3.1GHz,8,256.0,Intel Iris Plus Graphics 650,macOS,1.37,1633.628736
...,...,...,...,...,...,...,...,...,...,...,...
1298,Lenovo,2 in 1 Convertible,14.0,IPS Panel Full HD / Touchscreen 1920x1080,Intel Core i7 6500U 2.5GHz,4,128.0,Intel HD Graphics 520,Windows 10,1.80,577.874880
1299,Lenovo,2 in 1 Convertible,13.3,IPS Panel Quad HD+ / Touchscreen 3200x1800,Intel Core i7 6500U 2.5GHz,16,512.0,Intel HD Graphics 520,Windows 10,1.30,1357.734240
1300,Lenovo,Notebook,14.0,1366x768,Intel Celeron Dual Core N3050 1.6GHz,2,64.0,Intel HD Graphics,Windows 10,1.50,207.419040
1301,HP,Notebook,15.6,1366x768,Intel Core i7 6500U 2.5GHz,6,1.0,AMD Radeon R5 M330,Windows 10,2.19,692.000640


### Preliminary Data Analysis

(DESCRIPTION)

REMOVE IF UNNECESSARY

### Separation Into Train and Test Data

We separate the data into train data (which will be used to develop the model), and test data (which will be used to gauge the accuracy of the model). The test data will not be seen by the model until it comes the time to test how well it performs.

In [7]:
laptop_train, laptop_test = train_test_split(
    laptop_data,
    test_size=.25,   # Test data will be a quarter of the full data set, train the rest
)
X_train = laptop_train[["ScreenSize_Inches", "Memory_GB", "Storage", "Weight_Kg"]]
y_train = laptop_train["Price_CAD"]

X_test = laptop_test[["ScreenSize_Inches", "Memory_GB", "Storage", "Weight_Kg"]]
y_test = laptop_test["Price_CAD"]


In [8]:
laptop_train.head(5)

,Company,TypeName,ScreenSize_Inches,ScreenResolution,Cpu,Memory_GB,Storage,Gpu,OpSys,Weight_Kg,Price_CAD
241,Asus,Notebook,17.3,Full HD 1920x1080,Intel Core i7 8550U 1.8GHz,8,128.0,Nvidia GeForce 150MX,Windows 10,2.10,1037.09520
1080,Lenovo,Ultrabook,12.5,IPS Panel Touchscreen 2560x1440,Intel Core M 6Y75 1.2GHz,8,512.0,Intel HD Graphics 515,Windows 10,0.99,1267.15824
147,Asus,Notebook,15.6,Full HD 1920x1080,Intel Celeron Dual Core N3350 1.1GHz,4,1.0,Intel HD Graphics 500,Windows 10,2.00,311.58144
746,Samsung,Ultrabook,13.3,Full HD 1920x1080,Intel Core i7 7500U 2.7GHz,16,256.0,Intel HD Graphics 620,Windows 10,0.81,1493.59824
1270,Lenovo,2 in 1 Convertible,14.0,IPS Panel Full HD / Touchscreen 1920x1080,Intel Core i7 6500U 2.5GHz,4,128.0,Intel HD Graphics 520,Windows 10,1.80,577.87488


## Building the Model

To select the optimal value for k in k-nearest neighbors (k-NN) regression, we will utilize cross-validation on our training data. We will employ the Root Mean Squared Percentage Error (RMSPE) as the scoring metric. A smaller RMSPE indicates that the predicted values closely align with the true values, while a larger RMSPE suggests a larger deviation.

First, we will create a pipeline for k-NN regression. The pipeline will consist of the k-NN algorithm, along with the StandardScaler for standardizing the numerical features. We will store this pipeline in an object called "laptop_pipe". Next, we will perform cross-validation using the cross_validate function with 10 folds, specifying that we will use the negative RMSPE ("neg_root_mean_squared_error") as the scoring metric.

In [13]:
knn=KNeighborsRegressor()

laptop_pipe = make_pipeline(StandardScaler(),knn)

laptop_cv=pd.DataFrame(
    cross_validate(
        estimator=laptop_pipe,
        cv=10,
        X=X_train,
        y=y_train,
        scoring="neg_root_mean_squared_error",
        return_train_score=True,
    )
)
laptop_cv

,fit_time,score_time,test_score,train_score
0,0.007930,0.004725,-305.234685,-319.329553
1,0.009058,0.005445,-424.038323,-307.334961
2,0.007039,0.004260,-297.605291,-324.645483
3,0.007241,0.005329,-316.234790,-324.403210
4,0.006388,0.004796,-371.836831,-312.372239
5,0.007299,0.004311,-389.959378,-319.459551
6,0.006382,0.005345,-359.902260,-315.207774
7,0.008322,0.005228,-478.813340,-304.423813
8,0.007442,0.004427,-486.351961,-302.368197
9,0.006956,0.004627,-412.285067,-316.294989


We will now use a `param_grid` to test out 200 values of `k`, in order to find the most effective one to use for our model. We will employ the function `GridSearchCV`, passing the parameters `laptop_pipe`, `cv=10`, `n_jobs=-1` and once again `scoring="neg_root_mean_squared_error`.


In [14]:
param_grid = {"kneighborsregressor__n_neighbors": range(1, 201, 1)}
laptop_tuned = GridSearchCV(laptop_pipe, param_grid, cv=10, n_jobs=-1, scoring="neg_root_mean_squared_error")
laptop_results = pd.DataFrame(laptop_tuned.fit(X_train,y_train).cv_results_)
laptop_results

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_kneighborsregressor__n_neighbors,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
0,0.007095,0.000785,0.004497,0.000411,1,{'kneighborsregressor__n_neighbors': 1},-366.354006,-566.207155,-507.912990,-352.207890,-439.688967,-450.104452,-525.761779,-513.396457,-425.166230,-441.336595,-458.813652,65.725352,163
1,0.008476,0.002380,0.005060,0.000765,2,{'kneighborsregressor__n_neighbors': 2},-350.356481,-479.366096,-385.325331,-306.255345,-379.655262,-440.209775,-438.557007,-483.435669,-438.677215,-416.819978,-411.865816,53.494498,61
2,0.011276,0.006775,0.004788,0.000729,3,{'kneighborsregressor__n_neighbors': 3},-324.246478,-432.288571,-344.998681,-320.434125,-381.694261,-398.284843,-396.126447,-496.651245,-427.545658,-428.209854,-395.048016,52.096899,31
3,0.007409,0.000788,0.004691,0.000251,4,{'kneighborsregressor__n_neighbors': 4},-303.991217,-432.753207,-317.692089,-318.204644,-376.327705,-391.261737,-366.693030,-472.877580,-468.583928,-409.823710,-385.820885,57.967706,21
4,0.007848,0.000777,0.005602,0.000883,5,{'kneighborsregressor__n_neighbors': 5},-305.234685,-424.038323,-297.605291,-316.234790,-371.836831,-389.959378,-359.902260,-478.813340,-486.351961,-412.285067,-384.226193,63.947163,19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,0.006680,0.000313,0.008407,0.000320,196,{'kneighborsregressor__n_neighbors': 196},-420.309646,-476.313316,-378.800214,-408.105427,-396.803984,-528.770869,-461.207380,-628.747243,-517.841209,-545.647841,-476.254713,75.078066,196
196,0.006699,0.000212,0.008278,0.000173,197,{'kneighborsregressor__n_neighbors': 197},-420.533739,-476.477278,-379.367136,-408.458135,-397.539402,-529.140771,-461.879941,-628.966665,-518.471815,-546.117043,-476.695192,75.013434,197
197,0.006570,0.000211,0.008374,0.000219,198,{'kneighborsregressor__n_neighbors': 198},-420.967921,-476.897204,-379.652979,-408.904761,-397.797293,-529.748057,-462.811444,-629.786912,-518.740828,-546.455889,-477.176329,75.113046,198
198,0.006416,0.000180,0.008431,0.000269,199,{'kneighborsregressor__n_neighbors': 199},-421.380414,-477.399263,-380.602024,-408.904439,-397.819917,-530.174470,-463.286837,-630.516601,-519.255981,-547.328853,-477.666880,75.234931,199


And now we can determine the best parameter of `K` as well as the lowest RMSE produced.

In [20]:
laptop_param = laptop_tuned.best_params_
laptop_RMSE = -laptop_tuned.best_score_

laptop_param

{'kneighborsregressor__n_neighbors': 11}

In [21]:
laptop_RMSE

376.8411474729222

## Putting the Model to the Test

To evaluate the performance of our model in predicting unseen data, we will calculate the Root Mean Squared Percentage Error (RMSPE) on the test data. We will utilize the predict function to generate predictions on the test data using `laptop_tuned` (which automatically uses the best `k` parameter) and store the results in a variable called "laptop_prediction". Then, we will compute the RMSPE on the test data using the mean_squared_error function.

In [15]:
laptop_prediction = laptop_tuned.predict(X_test)
summary = mean_squared_error(laptop_prediction, y_test)**(1/2)

summary

386.2866224507617